<a href="https://colab.research.google.com/github/malkab/turismo_geo_ml/blob/main/notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
Test